In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool


# def create_localiser(L,rho,kappa,disorder):
#     # Function to create the localiser matrix of a 1D Anderson model
#     # Note the open boundary conditions
#     # This also includes the creation of the position operator X

#     # on diagonal disorder
#     diag = (np.random.rand(L)-0.5) * disorder
#     # off diagonal hopping terms
#     off_diag = np.ones(L-1)

#     H = sp.diags([off_diag,diag,off_diag],[-1,0,1],shape=(L,L),format='lil')

#     #H[L-1,0] = 1
#     #H[0,L-1] = 1

#     row_vector = np.linspace(-rho,rho,L)
#     X = sp.diags(row_vector,0,shape=(L,L),format='csr')

#     localiser = sp.bmat([[-H,kappa * X],[kappa * X,H]],format='csr')

#     return localiser


m = 1
S = 2


def create_localiser(L,rho,kappa,disorder):
    mass_term = m * np.ones(L//2)
    shift_term = S * np.ones(L//2 - 1)
    A = sp.diags([shift_term,mass_term],[-1,0],shape=(L//2,L//2),format='csr')
    H0 = sp.bmat([[sp.csr_matrix((L//2,L//2)),A.conj.T],[A,sp.csr_matrix((L//2,L//2))]],format='csr')
    disorder = (np.random.rand(L)-0.5) * disorder
    H_disorder = sp.diags(disorder,0,shape=(L//2,L//2),format='csr')
    H = H0 + sp.bmat([[sp.csr_matrix((L//2,L//2)),H_disorder],[H_disorder,sp.csr_matrix((L//2,L//2))]],format='csr')

    row_vector_A = sp.diags(np.linspace(-rho,(rho-(rho/(2*L))),L),0,shape=(L//2,L//2),format='csr')
    row_vector_B = sp.diags(np.linspace(((rho/(2*L) )-rho),rho,L),0,shape=(L//2,L//2),format='csr')

    X = sp.bmat([[row_vector_A,sp.csr_matrix((L//2,L//2))],[sp.csr_matrix((L//2,L//2)),row_vector_B]],format='csr')
    localiser = sp.bmat([[kappa * X, H.T],[H,-kappa * X]],format='csr')

    return localiser

def find_eigenvalues(localiser, num_eigenvalues=800):
    # Function to find the positive eigenvalues of the localiser matrix
    # numpy eigsh is best for the sparse case.
    eigvals, eigvecs = eigsh(localiser, k=num_eigenvalues,sigma = 0, which='LM')
    positive_eigvals = eigvals[eigvals > 0]
    return positive_eigvals



# Calculate the adjacent gap ratio r = min(s_i,s_(i+1))/max(s_i,s_(i+1))
def calculate_r(eigvals):
    # Once eigenvalues are found, calculate the r value
    eigvals_s = np.diff(eigvals)
    min_eigvals_s = np.array([min(eigvals_s[i],eigvals_s[i+1]) for i in range(len(eigvals_s)-1)])
    max_eigvals_s = np.array([max(eigvals_s[i],eigvals_s[i+1]) for i in range(len(eigvals_s)-1)])
    r = min_eigvals_s / max_eigvals_s
    return r.mean()


def calculate_z(eigvals):    
    eigvals = sorted(eigvals)
    z = np.zeros(len(eigvals)-4)
    for i in range(2,len(eigvals)-2):
        if abs(eigvals[i+1] - eigvals[i]) < abs(eigvals[i]-eigvals[i-1]):
            nn = abs(eigvals[i+1] - eigvals[i])
            nnn = min(min(abs(eigvals[i]-eigvals[i-1]),abs(eigvals[i+2]-eigvals[i])),abs(eigvals[i-2]-eigvals[i]))
        else:
            nn = abs(eigvals[i]-eigvals[i-1])
            nnn = min(min(abs(eigvals[i+1]-eigvals[i]),abs(eigvals[i-2]-eigvals[i])),abs(eigvals[i+2]-eigvals[i]))
        z[i-2] = nn/nnn
    return z.mean()
